In [1]:
import pandas as pd
import alphalens as al
import numpy as np
from datetime import datetime
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

## 因子计算

In [4]:
kline_data=pd.read_csv('kline_data_2021-2022.csv',index_col=0)

In [5]:
kline_data['date'] = kline_data['date'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))

In [6]:
kline_data.head()

,asset,market,open_price,high_price,low_price,close_price,turnover,volume,date
0,000001.SZ,stock,191000.0,191000.0,184400.0,186000.0,2.891682e+09,155421643.0,2021-01-04
1,000001.SZ,stock,184000.0,184800.0,178000.0,181700.0,3.284607e+09,182135210.0,2021-01-05
2,000001.SZ,stock,180800.0,195600.0,180000.0,195600.0,3.648522e+09,193494512.0,2021-01-06
3,000001.SZ,stock,195200.0,199800.0,192300.0,199000.0,3.111275e+09,158418530.0,2021-01-07
4,000001.SZ,stock,199000.0,201000.0,193100.0,198500.0,2.348316e+09,119547322.0,2021-01-08


In [7]:
price_data = (kline_data[['asset', 'close_price', 'date']].pivot_table(values='close_price', index='date', columns='asset')/10000).sort_index()
factor_data = price_data.pct_change(1).stack().reset_index()
factor_data.rename(columns={0:'factor'}, inplace=True)

In [8]:
factor_data.head()

,date,asset,factor
0,2021-01-05,000001.SZ,-0.023118
1,2021-01-05,000002.SZ,0.004680
2,2021-01-05,000004.SZ,-0.006660
3,2021-01-05,000005.SZ,-0.019841
4,2021-01-05,000006.SZ,-0.014493


In [9]:
demo = factor_data[((factor_data['date']=='2021-01-05')|(factor_data['date']=='2021-01-06'))&((factor_data['asset']=='000001.SZ')|(factor_data['asset']=='000002.SZ'))].sort_values(by='asset')

In [10]:
demo

,date,asset,factor
0,2021-01-05,000001.SZ,-0.023118
4335,2021-01-06,000001.SZ,0.076500
1,2021-01-05,000002.SZ,0.004680
4336,2021-01-06,000002.SZ,0.030097


## 数据整理

In [11]:
share_data = pd.read_csv('share_data_2021-2022.csv', index_col=False)
# share_data.rename(columns={'hjcode':'asset', 'trans_date':'date'}, inplace=True)
share_data['date'] = share_data['date'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
share_data.head()

,asset,date,circulation_a,total_a
0,000001.SZ,2021-01-04,1.940575e+10,19405918198
1,000002.SZ,2021-01-04,9.714315e+09,9724196533
2,000004.SZ,2021-01-04,8.391868e+07,165052625
3,000005.SZ,2021-01-04,1.057946e+09,1058536842
4,000006.SZ,2021-01-04,1.348308e+09,1349995046


#### 计算流通市值

In [8]:
kline_data = kline_data.merge(share_data, on=['asset', 'date'])
kline_data['tcap'] = kline_data['close_price'] * kline_data['circulation_a']/10000

factor_data = factor_data.merge(kline_data[['asset', 'date', 'tcap']], on=['asset', 'date'])
factor_data.head()

,date,asset,factor,tcap
0,2021-01-05,000001.SZ,-0.023118,3.526025e+11
1,2021-01-05,000002.SZ,0.004680,2.711265e+11
2,2021-01-05,000004.SZ,-0.006660,1.752222e+09
3,2021-01-05,000005.SZ,-0.019841,2.613127e+09
4,2021-01-05,000006.SZ,-0.014493,7.334795e+09


In [9]:
indus_data = pd.read_csv('indus_data_2021-2022.csv', index_col=False)
indus_data['date'] = indus_data['date'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))

In [10]:
indus_data.head()

,asset,first_industry_code,first_industry_name,date
0,000001.SZ,40,银行,2021-01-04
1,000002.SZ,42,房地产,2021-01-04
2,000004.SZ,35,医药,2021-01-04
3,000005.SZ,20,电力及公用事业,2021-01-04
4,000006.SZ,42,房地产,2021-01-04


In [11]:
factor_data = factor_data.merge(indus_data[['asset', 'date', 'first_industry_code']], on=['asset', 'date'])
factor_data.head()

,date,asset,factor,tcap,first_industry_code
0,2021-01-05,000001.SZ,-0.023118,3.526025e+11,40
1,2021-01-05,000002.SZ,0.004680,2.711265e+11,42
2,2021-01-05,000004.SZ,-0.006660,1.752222e+09,35
3,2021-01-05,000005.SZ,-0.019841,2.613127e+09,20
4,2021-01-05,000006.SZ,-0.014493,7.334795e+09,42


In [42]:
dummies_demo = factor_data[factor_data['date']=='2021-01-05'].copy()

In [43]:
dummies_demo.head()

,date,asset,factor,tcap,first_industry_code
0,2021-01-05,000001.SZ,-0.023118,3.526025e+11,40
1,2021-01-05,000002.SZ,0.004680,2.711265e+11,42
2,2021-01-05,000004.SZ,-0.006660,1.752222e+09,35
3,2021-01-05,000005.SZ,-0.019841,2.613127e+09,20
4,2021-01-05,000006.SZ,-0.014493,7.334795e+09,42


In [45]:
pd.get_dummies(dummies_demo['first_industry_code'])

,10,11,12,20,21,22,23,24,25,26,...,40,41,42,43,50,60,61,62,63,70
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4123,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 去极值，标准化和中性化

In [12]:
def winsorize(x, method):   
    '''去极值化函数，输入原始向量，输出去极值化后的factor、pct、cmv和industry（向量）'''
    
    # 先验去极值
    if method == 'factor':
        alpha = 0.002
        up = x.quantile(1 - alpha / 2)
        down = x.quantile(alpha / 2)
        x = np.where((x > down) & (x < up), x, np.nan)  #先验信息，0.1%和99.9%之外的factor值改为np.nan
        #x = np.where(x > 0, x, np.nan)  #对于某些财务因子，如PE和PB，去掉负值
    else:
        pass
    
    # 3-sigma去极值  
    mean = np.mean(x)
    std = np.std(x)
    result = np.where(x > mean+3*std, mean+3*std, x)
    result = np.where(x < mean-3*std, mean-3*std, result)
    return result

In [13]:
# 调整因子函数
def adjust(data):
    '''macbeth循环取残差函数'''
    
    # 筛选日期    
    data.set_index('date', inplace=True) #设置日期为索引
    data.sort_index(inplace=True) #排序

    # 设立新变量
    adjusted_factor = pd.DataFrame(columns=['asset', 'factor'])       
    days = sorted(set(data.index)) # 去重拿到所有的交易日

    for i, day in enumerate(days): 
        
        print (i, day)
        da = data.loc[day].copy()
        
        # 去极值化
        da['factor'] = winsorize(da['factor'], 'factor')   #去掉0到100之外的数值和3-sigma去极值
        da.dropna(inplace=True)  
        
        # 标准化
        da[['factor','tcap']] = ((da[['factor','tcap']] - da[['factor','tcap']].mean()) / da[['factor','tcap']].std()).values
        #da['cmv'] = ((da['cmv'] - da['cmv'].mean()) / da['cmv'].std()).values
        industry_day = pd.get_dummies(da['first_industry_code'])  #行业哑变量     

        # 回归
        X = np.column_stack((da['tcap'], industry_day))
        #X = industry_day
        X = sm.add_constant(X)
        model = sm.OLS(da['factor'], X)
        result = model.fit()
        #result.summary()
        da.loc[day, 'factor'] = result.resid    #残差值作为新的因子值    
        da = da[['asset', 'factor']].reset_index()
        if i == 0:
            adjusted_factor = da.copy()
        else:
            adjusted_factor =  pd.concat([adjusted_factor, da])
        
    data = data.reset_index()
    
        
    return adjusted_factor.merge(data[['asset','date']],how='left',on=['asset','date'])

In [14]:
adj_factor_data = adjust(factor_data.copy())

0 2021-01-05 00:00:00
1 2021-01-06 00:00:00
2 2021-01-07 00:00:00
3 2021-01-08 00:00:00
4 2021-01-11 00:00:00
5 2021-01-12 00:00:00
6 2021-01-13 00:00:00
7 2021-01-14 00:00:00
8 2021-01-15 00:00:00
9 2021-01-18 00:00:00
10 2021-01-19 00:00:00
11 2021-01-20 00:00:00
12 2021-01-21 00:00:00
13 2021-01-22 00:00:00
14 2021-01-25 00:00:00
15 2021-01-26 00:00:00
16 2021-01-27 00:00:00
17 2021-01-28 00:00:00
18 2021-01-29 00:00:00
19 2021-02-01 00:00:00
20 2021-02-02 00:00:00
21 2021-02-03 00:00:00
22 2021-02-04 00:00:00
23 2021-02-05 00:00:00
24 2021-02-08 00:00:00
25 2021-02-09 00:00:00
26 2021-02-10 00:00:00
27 2021-02-18 00:00:00
28 2021-02-19 00:00:00
29 2021-02-22 00:00:00
30 2021-02-23 00:00:00
31 2021-02-24 00:00:00
32 2021-02-25 00:00:00
33 2021-02-26 00:00:00
34 2021-03-01 00:00:00
35 2021-03-02 00:00:00
36 2021-03-03 00:00:00
37 2021-03-04 00:00:00
38 2021-03-05 00:00:00
39 2021-03-08 00:00:00
40 2021-03-09 00:00:00
41 2021-03-10 00:00:00
42 2021-03-11 00:00:00
43 2021-03-12 00:00:0

In [15]:
adj_factor_data.head()

,date,asset,factor
0,2021-01-05,000001.SZ,-0.728113
1,2021-01-05,000002.SZ,0.057418
2,2021-01-05,000004.SZ,-0.446575
3,2021-01-05,000005.SZ,-0.047404
4,2021-01-05,000006.SZ,-0.295569


## 因子正交

In [16]:
# 生成几个时间截面样例因子
temp_factor_data = factor_data.loc[factor_data['date'] == '2021-01-05', ['date', 'asset', 'factor', 'tcap']].copy().fillna(0)

In [17]:
temp_factor_data.set_index(['date', 'asset'], inplace=True)

In [18]:
temp_factor_data['x2'] = temp_factor_data['factor']**2

In [19]:
temp_factor_data.corr()

,factor,tcap,x2
factor,1.000000,0.091117,0.41840
tcap,0.091117,1.000000,0.02565
x2,0.418400,0.025650,1.00000


#### 施密特正交

In [20]:
from sympy import *

In [21]:
a = GramSchmidt([Matrix(temp_factor_data.values[:, i]) for i in range(len(temp_factor_data.T.values))],orthonormal=False)

In [22]:
len(a[0])

4125

In [23]:
schmidt_temp_factor_data = pd.DataFrame(np.column_stack((np.array(a[0]), np.array(a[1]) , np.array(a[2]))), index=temp_factor_data.index, columns=temp_factor_data.columns)
schmidt_temp_factor_data = schmidt_temp_factor_data.astype(float)

In [24]:
schmidt_temp_factor_data.head()

factor          tcap        x2
date       asset                                      
2021-01-05 000001.SZ -0.023118  3.570016e+11  0.000319
           000002.SZ  0.004680  2.702361e+11 -0.000898
           000004.SZ -0.006660  3.019581e+09  0.000263
           000005.SZ -0.019841  6.388620e+09  0.001053
           000006.SZ -0.014493  1.009255e+10  0.000677

In [25]:
# 施密特正交之后各向量之间相关系数
schmidt_temp_factor_data.corr()

,factor,tcap,x2
factor,1.000000,0.004064,0.007151
tcap,0.004064,1.000000,-0.094622
x2,0.007151,-0.094622,1.000000


#### 对称正交

In [26]:
# 求解特征值和特征向量
D, U = np.linalg.eig(np.dot(temp_factor_data.T, temp_factor_data))  

In [27]:
D

array([2.08343718e+25, 4.12007423e+00, 2.76653478e-02])

In [28]:
U = np.mat(U)
U

matrix([[ 3.83819238e-14,  9.99427938e-01, -3.37953094e-02],
        [ 1.00000000e+00, -3.80543487e-14, -2.80237682e-15],
        [ 4.08683218e-15,  3.38200615e-02,  9.99428775e-01]])

$F_{hat} = F * S$  
$S=U*D^{**(-0.5)}*U^{T}$

In [29]:

d = np.diag(D**(-0.5))
S = U*d*U.T 
#F_hat = np.dot(factors, S) 
F_hat = np.mat(temp_factor_data)*S 
factors_orthogonal = pd.DataFrame(F_hat, columns=temp_factor_data.columns, index=temp_factor_data.index)  

In [30]:
factors_orthogonal.describe()

,factor,tcap,x2
count,4125.000000,4.125000e+03,4125.000000
mean,-0.000753,3.481430e-03,0.005861
std,0.015554,1.517761e-02,0.014425
min,-0.107817,-3.205975e-15,-0.045160
25%,-0.008945,4.665201e-04,-0.000510
50%,-0.002651,9.026821e-04,0.002009
75%,0.005370,2.131567e-03,0.006486
max,0.092689,5.667860e-01,0.279845


In [31]:
factors_orthogonal.corr()

,factor,tcap,x2
factor,1.000000,0.010930,0.020028
tcap,0.010930,1.000000,-0.093152
x2,0.020028,-0.093152,1.000000


In [32]:
temp = temp_factor_data.head(10).copy()
temp.corr()

,factor,tcap,x2
factor,1.000000,0.084303,-0.927127
tcap,0.084303,1.000000,-0.157102
x2,-0.927127,-0.157102,1.000000


In [35]:
def calculate_corr(window):
    return np.corrcoef(window['factor'], window['tcap'])[0, 1]

In [37]:
temp.rolling(2).apply(calculate_corr, raw=True)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

kline_data = pd.read_csv('kline_data_2021-2022.csv', index_col=False)
kline_data['date'] = kline_data['date'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
kline_data['open_price']=kline_data['open_price']/10000
kline_data['close_price']=kline_data['close_price']/10000
kline_data['high_price']=kline_data['high_price']/10000
kline_data['low_price']=kline_data['low_price']/10000

kline_data = kline_data.rename(columns={
        "date": "datetime", 
        "asset": "sec_code","open_price": "open","close_price": "close","high_price": "high","low_price": "low"})
kline_data["openinterest"] = 0
daily_price=kline_data[['sec_code','datetime', "open", "close", "high", "low", "volume", 'openinterest']]
daily_price=daily_price.set_index('datetime')
daily_price.head()

,sec_code,open,close,high,low,volume,openinterest
datetime,,,,,,,
2021-01-04,000001.SZ,19.10,18.60,19.10,18.44,155421643.0,0
2021-01-05,000001.SZ,18.40,18.17,18.48,17.80,182135210.0,0
2021-01-06,000001.SZ,18.08,19.56,19.56,18.00,193494512.0,0
2021-01-07,000001.SZ,19.52,19.90,19.98,19.23,158418530.0,0
2021-01-08,000001.SZ,19.90,19.85,20.10,19.31,119547322.0,0


In [8]:
import alphalens as al
from datetime import datetime
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

df_2 = kline_data[['datetime', 'sec_code', "close"]]
df_2['datetime'] = pd.to_datetime(df_2['datetime'])
# print(df_all)

close = df_2.pivot(index='datetime', columns='sec_code', values='close')


# 读取已经计算好的因子
alpha = pd.read_csv('icmean_date_asset.csv')
alpha = alpha.rename(columns={
        "synthesized_factor": "factor"})
alpha['date'] = alpha['date'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
alpha = alpha.set_index(['date', 'asset'])['factor']

# 因子矩阵转换为一维数据(alphalens需要的格式)
# alpha = alpha.melt(id_vars=['date'], var_name='asset', value_name='factor' )

# date列转为日期格式
# alpha['date'] = pd.to_datetime(alpha['date'])
# alpha = alpha[['date', 'asset', 'factor']]

# 设置二级索引
# alpha = alpha.set_index(['date', 'asset'], drop=True)
# alpha.sort_index(inplace=True)


ret = get_clean_factor_and_forward_returns(alpha, close,quantiles=5)
ret = ret.reset_index()
ret = ret[ret['factor_quantile'] == 5]
ret = ret[['date','asset']]
ret['weight'] = 1/60
trade_info = ret.rename(columns={
        "date": "trade_date", 
        "asset": "sec_code"})
trade_info

C:\Users\任亦冉\AppData\Local\Temp\ipykernel_16020\3733421031.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['datetime'] = pd.to_datetime(df_2['datetime'])
E:\anaconda\envs\alphatest\lib\site-packages\alphalens\utils.py:319: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  delta_days = period_len.components.days - mode(days_diffs).mode[0]
E:\anaconda\envs\alphatest\lib\site-package

Dropped 4.1% entries from factor data: 4.1% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


,trade_date,sec_code,weight
5,2021-01-06,600531.SH,0.016667
14,2021-01-06,600545.SH,0.016667
23,2021-01-06,600558.SH,0.016667
26,2021-01-06,600561.SH,0.016667
44,2021-01-06,600505.SH,0.016667
...,...,...,...
902141,2021-12-17,003023.SZ,0.016667
902143,2021-12-17,003026.SZ,0.016667
902144,2021-12-17,003027.SZ,0.016667
902145,2021-12-17,003028.SZ,0.016667
